# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [2]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test
from DataLoader import generar_rolling_features,crear_archivo_kaggle,generar_lag_features
import xgboost as xgb
import pandas as pd

In [ ]:
ventas=importar_ventas()

,date,demand,mean_price,std_price,max_price,min_price,mean_discount,mean_diff_factory,std_discount,max_discount,...,min_diff_factory,size,group,category,cluster,store_cod,subgroup_cod,day,month,year
0,2021-01-10,1.000000,319.510000,0.000000,319.51,319.51,-0.017057,-0.240148,0.000000,-0.017057,...,-0.240148,1.0,Computing,Electronics,0,0,0,10,1,2021
1,2021-01-13,8.000041,241.460000,0.000000,241.46,241.46,-0.060742,-0.529973,0.038716,-0.022778,...,-0.993703,3.0,Computing,Electronics,0,0,0,13,1,2021
2,2021-01-18,4.000000,250.180000,0.000000,250.18,250.18,0.012871,-0.169803,0.000000,0.012871,...,-0.169803,4.0,Computing,Electronics,0,0,0,18,1,2021
3,2021-01-22,5.000000,268.270000,0.000000,268.27,268.27,0.029215,-0.187797,0.010120,0.046744,...,-0.300475,4.0,Computing,Electronics,0,0,0,22,1,2021
4,2021-01-23,3.000000,297.770000,0.000000,297.77,297.77,0.010243,-0.113674,0.000000,0.010243,...,-0.113674,3.0,Computing,Electronics,0,0,0,23,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206788,2023-12-15,2.000000,27.320000,0.000000,27.32,27.32,0.067716,-0.106606,0.000000,0.067716,...,-0.106606,2.0,Games,Toys,8,156,73,15,12,2023
5206789,2023-12-16,15.000593,29.456667,3.035845,33.75,27.31,-0.044976,-0.403494,0.038643,-0.008056,...,-0.570251,6.0,Games,Toys,8,156,73,16,12,2023
5206790,2023-12-21,3.999622,26.450000,0.000000,26.45,26.45,0.049397,-0.180057,0.017459,0.074088,...,-0.270060,3.0,Games,Toys,8,156,73,21,12,2023
5206791,2023-12-26,2.000000,26.530000,0.000000,26.53,26.53,-0.006408,-0.233237,0.000000,-0.006408,...,-0.233237,2.0,Games,Toys,8,156,73,26,12,2023


In [5]:
ventas.isnull().sum()

date                      0
demand                11096
mean_price            11096
std_price            410903
max_price             11096
min_price             11096
mean_discount         11096
mean_diff_factory     11096
std_discount         410903
max_discount          11096
min_discount          11096
total_sales           11096
std_diff_factory     410903
max_diff_factory      11096
min_diff_factory      11096
size                  11096
group                     0
category                  0
cluster                   0
store_cod                 0
subgroup_cod              0
day                       0
month                     0
year                      0
dtype: int64

In [6]:
agregar_fourier(ventas,2)

In [7]:
ventas=generar_rolling_features(ventas,"mean_price",[1,2],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"total_sales",[1,2],["subgroup_cod","store_cod"])


In [8]:
 for i in range(1,8): #agregamos lag a 7 dias por que vamos a predecir 7 dias
	generar_lag_features(ventas,"total_sales",i)

In [9]:
#busquemos otra fecha para probar esto objetivo 01-01-2023 a 07-01-2023

# ventas_2023=ventas[ventas["date"]<"02-01-2023"]
# test_2023=ventas_2023.groupby(["subgroup_cod","store_cod"]).tail(7)
# train_2023=ventas_2023.drop(test_2023.index)


ventas.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'mean_discount', 'mean_diff_factory', 'std_discount', 'max_discount',
       'min_discount', 'total_sales', 'std_diff_factory', 'max_diff_factory',
       'min_diff_factory', 'size', 'group', 'category', 'cluster', 'store_cod',
       'subgroup_cod', 'day', 'month', 'year', 'anual_sin_1', 'anual_cos_1',
       'mensual_sin_1', 'mensual_cos_1', 'anual_sin_2', 'anual_cos_2',
       'mensual_sin_2', 'mensual_cos_2', 'mean_price_rolling_mean_7d',
       'mean_price_rolling_sum_7d', 'mean_price_rolling_mean_14d',
       'mean_price_rolling_sum_14d', 'total_sales_rolling_mean_7d',
       'total_sales_rolling_sum_7d', 'total_sales_rolling_mean_14d',
       'total_sales_rolling_sum_14d', 'total_sales_lag_1', 'total_sales_lag_2',
       'total_sales_lag_3', 'total_sales_lag_4', 'total_sales_lag_5',
       'total_sales_lag_6', 'total_sales_lag_7'],
      dtype='object')

In [13]:
import numpy as np
FEATURES=[
 #  'mean_price', 
   # 'cluster', 
   # 'store_cod', 
    'subgroup_cod',
   'total_sales_lag_1',
    #'total_sales_lag_2', 
   # 'total_sales_lag_3', #'total_sales_lag_4',
    #'total_sales_lag_5', 'total_sales_lag_6', 'total_sales_lag_7', 
    #'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
  'mensual_sin_1', 
  #  'mensual_cos_1',
 #  'anual_sin_2', 
 #   'anual_cos_2',
   'mensual_sin_2', 
 #   'mensual_cos_2',
    #"std_price",
   # "max_price", 
   #"min_price",
    'mean_price_rolling_mean_7d',
   # 'mean_price_rolling_sum_7d',
  #   'mean_price_rolling_mean_14d',
    'mean_price_rolling_sum_14d', 
   # 'mean_price_rolling_mean_21d',#  'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
    #  'mean_price_rolling_sum_28d', 
    #'demand_rolling_mean_7d',
    #  'demand_rolling_sum_7d', 
    #'demand_rolling_mean_14d',
     'total_sales_rolling_mean_7d',
       'total_sales_rolling_sum_7d', 
  #  'total_sales_rolling_mean_14d',
   #    'total_sales_rolling_sum_14d',
    #'std_discount', 'max_discount', 
    #'min_discount', 
    #'std_diff_factory',
    #"mean_discount",
    #"mean_diff_factory",
   # 'max_diff_factory', 
   # 'min_diff_factory',
    #"size"
]
#test=preparar_test("ids_test.csv",ventas,FEATURES)
TARGET=['total_sales']
ventas.fillna(0,inplace=True)
test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
#X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

#y_train_interno_log=np.log1p(y_train_interno)
#y_test_interno_log=np.log1p(y_test_interno)


# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

In [11]:
y_test_interno.describe(),y_train_interno.describe()

(        total_sales
 count  77672.000000
 mean     326.245738
 std      463.669729
 min        5.470000
 25%       85.100000
 50%      171.090000
 75%      353.430000
 max     6972.330000,
         total_sales
 count  5.129121e+06
 mean   3.551661e+02
 std    5.565812e+02
 min    0.000000e+00
 25%    8.328000e+01
 50%    1.744000e+02
 75%    3.698300e+02
 max    1.780270e+04)

In [12]:
ventas[ventas["total_sales"]>370][["demand","mean_price","max_price","total_sales"]]

,demand,mean_price,max_price,total_sales
161624,2.000000,268.89000,268.89,537.78
5125064,19.000844,27.40000,32.60,544.42
443642,3.999961,257.49000,257.49,1029.95
1185319,12.001095,45.68000,45.68,548.21
466964,5.000000,259.55000,259.55,1297.75
...,...,...,...,...
872693,15.000000,31.63000,31.63,474.45
508345,2.999966,293.72000,293.72,881.15
3476323,9.000228,44.39875,45.28,399.07
542205,3.000000,289.76000,289.76,869.28


## Modelo interno

In [15]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# ===== 1. CARGA DE DATOS =====
# Reemplazá por tu archivo/dataset procesado

df = ventas #pd.read_csv("ventas_final.csv", parse_dates=["date"])

TARGET = "total_sales"
#FEATURES = [col for col in df.columns if col not in [TARGET, "date"]]

# ===== 2. VALIDACIÓN TEMPORAL =====
fecha_corte = "2023-11-01"
mask_train = df["date"] < fecha_corte
mask_val   = df["date"] >= fecha_corte

X_train, X_val = df.loc[mask_train, FEATURES], df.loc[mask_val, FEATURES]
y_train, y_val = df.loc[mask_train, TARGET], df.loc[mask_val, TARGET]

print(f"Train: {X_train.shape}, Val: {X_val.shape}")
# ===== 3. MODELO XGB CON ANTI-OVERFIT =====
reg = XGBRegressor(
     n_estimators=500,           # alto para usar early stopping
    learning_rate=0.05,          # más bajo para no memorizar rápido
    max_depth=4,                 # menos profundidad => menos overfit
    #min_child_weight=5,          # evita que splits se hagan con poca data
    #subsample=0.8,                # usa solo parte de las filas en cada árbol
    colsample_bytree=1,         # usa solo parte de las columnas en cada árbol
    reg_lambda=1,                 # L2 regularization
    reg_alpha=20,                  # L1 regularization
   # gamma=0.2,                   # penaliza splits innecesarios
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=50,
    objetive='reg:squarederror',
    eval_metric="rmse"
)


# ===== 4. ENTRENAMIENTO CON EARLY STOPPING =====
reg.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=50,
)

# ===== 5. EVALUACIÓN =====
y_pred = reg.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE validación: {rmse:.4f}")

# ===== 6. IMPORTANCIA DE FEATURES =====
importances = pd.DataFrame({
    "feature": FEATURES,
    "importance": reg.feature_importances_
}).sort_values("importance", ascending=False)

print("\nImportancia de features:")
print(importances)

# ===== 7. GUARDAR RESULTADOS =====
importances.to_csv("importancia_features.csv", index=False)

Train: (4916559, 11), Val: (290234, 11)


C:\Users\emili\miniconda3\envs\labo\Lib\site-packages\xgboost\callback.py:386: UserWarning: [18:54:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "objetive" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	validation_0-rmse:511.89967
[50]	validation_0-rmse:353.41923
[100]	validation_0-rmse:348.09638
[150]	validation_0-rmse:346.96298
[200]	validation_0-rmse:346.48812
[250]	validation_0-rmse:346.27085
[300]	validation_0-rmse:346.16911
[350]	validation_0-rmse:346.11852
[400]	validation_0-rmse:346.04074
[450]	validation_0-rmse:345.96813
[499]	validation_0-rmse:345.94762
RMSE validación: 345.9476

Importancia de features:
                        feature  importance
0                  subgroup_cod    0.451857
7    mean_price_rolling_mean_7d    0.220905
9   total_sales_rolling_mean_7d    0.218131
10   total_sales_rolling_sum_7d    0.029065
2                          year    0.023446
6                 mensual_sin_2    0.019268
3                   anual_sin_1    0.018125
1             total_sales_lag_1    0.008415
4                   anual_cos_1    0.006185
5                 mensual_sin_1    0.003000
8    mean_price_rolling_sum_14d    0.001602


C:\Users\emili\miniconda3\envs\labo\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [18]:
pd.DataFrame(reg.feature_importances_,columns=["importance"],index=reg.feature_names_in_).sort_values(by="importance",ascending=False)

,importance
mean_price,0.281229
size,0.152627
total_sales_rolling_mean_7d,0.146224
subgroup_cod,0.109401
std_diff_factory,0.088605
min_diff_factory,0.075572
std_discount,0.045691
max_discount,0.026220
total_sales_rolling_sum_7d,0.015869
mean_diff_factory,0.014093


## Modelo kraggel


In [10]:
from sklearn.metrics import mean_squared_error
import numpy as np

prediccion=reg.predict(X_test_interno)


# y_true_log y y_pred_log están en escala log1p
y_true_original = y_test_interno
y_pred_original = prediccion
from sklearn.metrics import mean_squared_error
rmse_original = np.sqrt(mean_squared_error(y_true_original, y_pred_original))

print(rmse_original)
print(r2_score(y_true_original,y_pred_original))

71.42922014742395
0.9762676954269409


In [29]:
X_train=train[FEATURES]
y_train=train[TARGET]

reg = xgb.XGBRegressor(
    n_estimators=200,
    eval_metric="rmse",
    eta=0.05,
    max_depth=6,
   #colsample_bytree=0.9, #reducir el numero de columnas aumenta la importancia de std_price
    
)

reg.fit(
    X_train, y_train
)

prediccion=reg.predict(X_test_real)

kaggle=crear_archivo_kaggle(prediccion,test)

In [8]:
kaggle.isnull().sum()
# kaggle.loc[kaggle["TOTAL SALES"].isnull()]
# kaggle[['store_id', 'subgroup', 'date']] = kaggle["STORE_SUBGROUP_DATE_ID"].str.split('_', expand=True)
# nulls=kaggle.loc[kaggle["TOTAL SALES"].isnull()]

STORE_SUBGROUP_DATE_ID    0
TOTAL SALES               0
dtype: int64

In [78]:
nulls["subgroup"].unique()

array(['Basketball'], dtype=object)

In [29]:
test[['store_id', 'subgroup', 'date']] = test["store_subgroup_date_id"].str.split('_', expand=True)
test[test["subgroup"]=="Basketball"]






,date,store_subgroup_date_id,store_cod,subgroup_cod,mean_price,cluster,year,day,month,std_price,...,anual_sin_1,anual_cos_1,mensual_sin_1,mensual_cos_1,anual_sin_2,anual_cos_2,mensual_sin_2,mensual_cos_2,store_id,subgroup
33600,2024-01-01,S00001_Basketball_2024-01-01,0,32,NaN,0,2024,1,1,NaN,...,0.017202,0.999852,0.20495,0.978773,0.034398,0.999408,0.401198,0.915991,S00001,Basketball
33601,2024-01-02,S00001_Basketball_2024-01-02,0,32,NaN,0,2024,2,1,NaN,...,0.034398,0.999408,0.20495,0.978773,0.068755,0.997634,0.401198,0.915991,S00001,Basketball
33602,2024-01-03,S00001_Basketball_2024-01-03,0,32,NaN,0,2024,3,1,NaN,...,0.051584,0.998669,0.20495,0.978773,0.103031,0.994678,0.401198,0.915991,S00001,Basketball
33603,2024-01-04,S00001_Basketball_2024-01-04,0,32,NaN,0,2024,4,1,NaN,...,0.068755,0.997634,0.20495,0.978773,0.137185,0.990545,0.401198,0.915991,S00001,Basketball
33604,2024-01-05,S00001_Basketball_2024-01-05,0,32,NaN,0,2024,5,1,NaN,...,0.085906,0.996303,0.20495,0.978773,0.171177,0.985240,0.401198,0.915991,S00001,Basketball
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34645,2024-01-03,S00157_Basketball_2024-01-03,156,32,NaN,8,2024,3,1,NaN,...,0.051584,0.998669,0.20495,0.978773,0.103031,0.994678,0.401198,0.915991,S00157,Basketball
34646,2024-01-04,S00157_Basketball_2024-01-04,156,32,NaN,8,2024,4,1,NaN,...,0.068755,0.997634,0.20495,0.978773,0.137185,0.990545,0.401198,0.915991,S00157,Basketball
34647,2024-01-05,S00157_Basketball_2024-01-05,156,32,NaN,8,2024,5,1,NaN,...,0.085906,0.996303,0.20495,0.978773,0.171177,0.985240,0.401198,0.915991,S00157,Basketball
34648,2024-01-06,S00157_Basketball_2024-01-06,156,32,NaN,8,2024,6,1,NaN,...,0.103031,0.994678,0.20495,0.978773,0.204966,0.978769,0.401198,0.915991,S00157,Basketball


In [30]:
test.isnull().sum()

date                              0
store_subgroup_date_id            0
store_cod                         0
subgroup_cod                      0
mean_price                     1050
cluster                           0
year                              0
day                               0
month                             0
std_price                      1050
max_price                      1050
min_price                      1050
mean_price_rolling_mean_7d     1050
mean_price_rolling_sum_7d      1050
mean_price_rolling_mean_14d    1050
mean_price_rolling_sum_14d     1050
mean_price_rolling_mean_21d    1050
mean_price_rolling_sum_21d     1050
mean_price_rolling_mean_28d    1050
mean_price_rolling_sum_28d     1050
demand_rolling_mean_7d         1050
demand_rolling_sum_7d          1050
demand_rolling_mean_14d        1050
demand_rolling_sum_14d         1050
demand_rolling_mean_21d        1050
demand_rolling_sum_21d         1050
demand_rolling_mean_28d        1050
demand_rolling_sum_28d      

In [35]:
trans=leer_csv("eci_transactions")

trans[trans["subgroup"]=="Basketball"]

KeyboardInterrupt: 

In [67]:
test=preparar_test("ids_test.csv")

kaggle=DataLoader.crear_archivo_kaggle(prediccion,test)

In [68]:
kaggle.isnull().sum()

STORE_SUBGROUP_DATE_ID       0
TOTAL SALES               1050
dtype: int64

,DATE,STORE_SUBGROUP_DATE_ID,STORE_COD,SUBGROUP_COD,MEAN_PRICE,CLUSTER,YEAR,DAY,MONTH,STD_PRICE,...,DEMAND_ROLLING_SUM_21D,DEMAND_ROLLING_MEAN_28D,DEMAND_ROLLING_SUM_28D,STD_DISCOUNT,MAX_DISCOUNT,MIN_DISCOUNT,STD_DIFF_FACTORY,MAX_DIFF_FACTORY,MIN_DIFF_FACTORY,TOTAL SALES
0,2024-01-01,S00001_Laptops_2024-01-01,0,0,252.82,0,2024,1,1,0.0,...,94.000089,4.607139,128.999890,2.163747,9.18,4.59,14.820958,-31.44,-62.88,969.419743
1,2024-01-02,S00001_Laptops_2024-01-02,0,0,272.99,0,2024,2,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
2,2024-01-03,S00001_Laptops_2024-01-03,0,0,272.99,0,2024,3,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
3,2024-01-04,S00001_Laptops_2024-01-04,0,0,272.99,0,2024,4,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
4,2024-01-05,S00001_Laptops_2024-01-05,0,0,272.99,0,2024,5,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77695,2024-01-03,S00157_Puzzles_2024-01-03,156,73,26.53,8,2024,3,1,0.0,...,104.001067,4.428645,124.002064,0.000000,0.17,0.17,0.000000,-8.07,-8.07,70.690697
77696,2024-01-04,S00157_Puzzles_2024-01-04,156,73,30.32,8,2024,4,1,0.0,...,105.001067,4.285765,120.001425,0.000000,3.31,3.31,0.000000,-9.40,-9.40,82.628069
77697,2024-01-05,S00157_Puzzles_2024-01-05,156,73,30.32,8,2024,5,1,0.0,...,105.001067,4.285765,120.001425,0.000000,3.31,3.31,0.000000,-9.40,-9.40,82.628069
77698,2024-01-06,S00157_Puzzles_2024-01-06,156,73,30.32,8,2024,6,1,0.0,...,105.001067,4.285765,120.001425,0.000000,3.31,3.31,0.000000,-9.40,-9.40,82.628069
